In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/3', creation_time=1713784164751, experiment_id='3', last_update_time=1713784164751, lifecycle_stage='active', name='Housing2', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/2', creation_time=1713779782439, experiment_id='2', last_update_time=1713779782439, lifecycle_stage='active', name='Pricing1', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/1', creation_time=1713779776072, experiment_id='1', last_update_time=1713779776072, lifecycle_stage='active', name='Pricing', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/0', creation_time=1713779775998, experiment_id='0', last_update_time=1713779775998, lifecycle_stage='active', name='Default', tags={}>

In [4]:
client.create_experiment(name='file')

'4'

In [26]:
from mlflow.entities import ViewType
import mlflow
import pandas as pd
from sklearn.metrics import mean_squared_error
import pickle

In [7]:
runs = client.search_runs(
    experiment_ids='3',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [10]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5f648d64d3974ca5891443cbde573bf3, rmse: 25996.1931
run id: e47a42f7a9894aeba9b991b9fa4c1ac6, rmse: 25996.1931
run id: 9d6de6c127f442cca122469cb169fd86, rmse: 25996.1931
run id: e971293bdd1742f79d004dccddd8ba2d, rmse: 25996.1931
run id: cf03d113f2de4428833adc7d03f305da, rmse: 38733.5256


In [14]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = 'cf03d113f2de4428833adc7d03f305da'
model_uri =f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri, name='new')

Successfully registered model 'new'.
Created version '1' of model 'new'.


<ModelVersion: aliases=[], creation_timestamp=1713798593422, current_stage='None', description=None, last_updated_timestamp=1713798593422, name='new', run_id='cf03d113f2de4428833adc7d03f305da', run_link=None, source='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/3/cf03d113f2de4428833adc7d03f305da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [17]:
model_name = 'new'
latest_version = client.get_latest_versions(name=model_name)

for version in latest_version:
  print(f'version = {version.version}, stage = {version.current_stage}')

version = 1, stage = None


C:\Users\Victoria\AppData\Local\Temp\ipykernel_15604\3029521980.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.3/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(name=model_name)


In [19]:
model_version = 1
new_stage = 'staging'
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage='staging',
    archive_existing_versions=False
)

C:\Users\Victoria\AppData\Local\Temp\ipykernel_15604\153091783.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.3/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1713798593422, current_stage='Staging', description=None, last_updated_timestamp=1713799183348, name='new', run_id='cf03d113f2de4428833adc7d03f305da', run_link=None, source='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/3/cf03d113f2de4428833adc7d03f305da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [21]:
from datetime import datetime

date= datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f'This model version{model_version} was transitioned to {new_stage} on {date}'
)

<ModelVersion: aliases=[], creation_timestamp=1713798593422, current_stage='Staging', description='This model version1 was transitioned to staging on 2024-04-22', last_updated_timestamp=1713799552325, name='new', run_id='cf03d113f2de4428833adc7d03f305da', run_link=None, source='file:///c:/Users/Victoria/gt/MLOp/Housing_Prediction/experiments_tracking/mlruns/3/cf03d113f2de4428833adc7d03f305da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [33]:
df = pd.read_csv('C:/Users/Victoria/gt/MLOp/Housing_Prediction/Training.csv')

In [34]:
def read_dataframe(filename):
  df = pd.read_csv(filename)
  df = df.drop("Id", axis=1)
  df = df.drop(columns=["Alley", "PoolQC", "Fence", "MiscFeature"])

  df = df[df.Neighborhood == "Avondale"]
  df = df[(df.SalePrice >= 100000) & (df.SalePrice <= 500000)]

  categorical = ["BedroomAbvGr", "GarageCars"]
  df[categorical].astype(str)

  return df

def preprocess(df, dv):
  df["BED_GAR"] = df["BedroomAbvGr"].astype(str) + "_" + df["GarageCars"].astype(str)
  categorical = ["BedroomAbvGr", "GarageCars"]
  numerical = ["GarageArea"]
  train_dict = df[categorical + numerical].to_dict(orient="records")
  dv.fit_transform(train_dict)

def test_model(name, stage, X_test, y_test):
  model = mlflow.pyfunc.load_model(f'models:/{name}/{stage}')
  y_pred = model.predict(X_test)
  return {'rmse': mean_squared_error(y_test, y_pred, squared=False)}

In [35]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'c:\\Users\\Victoria\\gt\\MLOp\\Housing_Prediction\\experiments_tracking\\preprocessor'

In [36]:
with open('preprocessor/preprocessor.b', 'rb') as f_in:
  dv = pickle.load(f_in)

In [37]:
X_test = preprocess(df, dv)

In [38]:
target = 'SalePrice'
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage='Staging', X_test=X_test, y_test=y_test)